In [ ]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
# from azure.devops.v7_1.build.models import BuildStatus

# Set your Azure DevOps organization URL and personal access token
personal_access_token = ''
organization_url = 'https://dev.azure.com/noobDevOps'

# Set the project and pipeline information
project = 'TheMcguffin'
pipeline_id = "2"

# Create a connection to Azure DevOps
credentials = BasicAuthentication("", personal_access_token)
connection = Connection(base_url=organization_url, creds=credentials)

# Get the build client
build_client = connection.clients.get_core_client()
print(build_client,"\n\n")
[build for build in build_client if build.status == "Succeeded"]

# Define a function to get the total number of builds for a pipeline
# def get_total_builds_for_pipeline(project, pipeline_id):
#     # Create a build query
#     build_query = {
#         "definitions": [pipeline_id],
#         "status_filter": [BuildStatus.completed]
#     }

#     # Call the API to get the builds
#     builds = build_client.get_builds(project=project, query_order="finishTimeDescending", query=build_query)

#     # Get the total count of builds
#     total_builds = len(builds)

#     return total_builds

# Call the function to get the total builds for the pipeline
# total_builds = get_total_builds_for_pipeline(project, pipeline_id)
# print(f"Total builds for pipeline {pipeline_id}: {total_builds}")

In [ ]:
import azure.devops.core.credentials
import azure.devops.core.pipelines

# Create a credentials object
credentials = azure.devops.core.credentials.AccessTokenCredentials(
    "https://dev.azure.com/noobDevOps", ""
)

# Create a pipelines client
client = azure.devops.core.pipelines.PipelinesClient(credentials)

# Get the pipeline ID
pipeline_id = "your_pipeline_id"

# Get the builds for the pipeline
builds = client.get_builds(pipeline_id)

# Print the builds
for build in builds:
    print(build.id, build.name, build.status)


In [ ]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import matplotlib.pyplot as plt

def generate_graph():
    # Azure DevOps credentials
    personal_access_token = ''
    organization_url = 'https://dev.azure.com/noobDevOps'
    project_name = 'TheMcguffin'

    # Connect to Azure DevOps
    credentials = BasicAuthentication('', personal_access_token)
    connection = Connection(base_url=organization_url, creds=credentials)

    # Get the build client
    build_client = connection.clients.get_build_client()

    # Get build definitions
    definitions = build_client.get_definitions(project=project_name)

    images=[]
    results=[]
    
    for definition in definitions:
        success_count = 0
        failure_count = 0
        builds = build_client.get_builds(project=project_name, definitions=[definition.id])
        # builds = build_client.get_builds(project=project_name, definitions=[definition.id], min_time=min_time, max_time=(min_time + datetime.timedelta(days=1)).date().isoformat())
        for build in builds:
            if build.result == 'succeeded':
                success_count += 1
            elif build.result == 'failed':
                failure_count += 1
                
        results.append({
        'id': definition.id,
        'success': success_count,
        'failure': failure_count,
        })
        
        graph_image=f"{definition.id}_graph.png"
        images.append({
            'name': 'Bar Graph',
            'url': graph_image,
            'alt': 'Build Pipeline Bar Graph'
        })

        # Save the pie chart as an image
        pie_image=f"{definition.id}_piechart.png"
        images.append({
            'name': 'Pie Chart',
            'url': pie_image,
            'alt': 'Build Pipeline Pie Chart'
        })
    
    print(results,"\n\n")
    print(images,"\n\n")
    return len(results), len(images)

generate_graph()

In [ ]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication

# Azure DevOps credentials
personal_access_token = ''
organization_url = 'https://dev.azure.com/noobDevOps'
project_name = 'TheMcguffin'
repository_id = 'Second-Pipeline-yaml'

# Connect to Azure DevOps
credentials = BasicAuthentication('', personal_access_token)
connection = Connection(base_url=organization_url, creds=credentials)

# Get the git client
git_client = connection.clients.get_git_client()

# Get the repository
repository = git_client.get_repository(project=project_name, repository_id=repository_id)

# Get the total number of code commits
commit_count = git_client.get_all_commits(project=project_name, repository_id=repository_id).count

print(f"Total number of code commits: {commit_count}")


In [ ]:
import requests

# Azure DevOps credentials
personal_access_token = ''
organization_url = 'https://dev.azure.com/noobDevOps'
project_name = 'TheMcguffin'
repository_id = 'Second-Pipeline-yaml'

# API endpoint
api_url = f"{organization_url}/{project_name}/_apis/git/repositories/{repository_id}/commits?api-version=6.0"

# Request headers
headers = {
    'Authorization': f'Bearer {personal_access_token}',
    'Content-Type': 'application/json'
}

# Send GET request to retrieve commits
response = requests.get(api_url, headers=headers)

# Check if request was successful
if response.status_code == 200:
    commit_count = len(response.json()['value'])
    print(f"Total number of code commits: {commit_count}")
else:
    print(f"Failed to retrieve commits. Error: {response.text}")


In [ ]:
import requests
import base64
import json

# Get your personal access token (PAT) from Azure DevOps.
pat = ''

# Create the authorization header.
authorization = str(base64.b64encode(bytes(':' +pat, 'ascii')), 'ascii')

# Create the request headers.
headers = {
    'Accept': 'application/json',
    'Authorization': 'Basic ' + authorization
}

# Get the URL for the commits endpoint.
url = 'https://dev.azure.com/{organization}/{project}/_apis/git/repositories/{repositoryId}/commits?api-version=6.0'.format(
    organization='https://dev.azure.com/noobDevOps',
    project='TheMcguffin',
    repositoryId=1
)

# Make the request.
response = requests.get(url, headers=headers)

# Check the response status code.
if response.status_code == 200:
    # The request was successful.
    data = json.loads(response.content)
    print('The total number of commits is {}.'.format(data['count']))
else:
    # The request failed.
    print('The request failed with status code {}.'.format(response.status_code))


# For a specific pipeline

In [25]:
import requests
import json
import base64
from bs4 import BeautifulSoup
from datetime import datetime


def getPipelinesRuns():
    # Azure DevOps credentials
    pat = ''
    organization = 'noobDevOps'
    project = 'TheMcguffin'

    authorization = str(base64.b64encode(bytes(':'+pat, 'ascii')), 'ascii')

    print(authorization)

    url = f"https://dev.azure.com/{organization}/{project}/_apis/pipelines/1/runs?api-version=6.1-preview.1"
    # url = f"https://dev.azure.com/{organization}/{project}/_apis/pipelines?api-version=6.1-preview.1"

#     payload = json.dumps({})
    headers = {
      'Authorization': 'Basic '+authorization,
      'Content-Type': 'application/json',
    }

    response = requests.request("GET", url, headers=headers)

    current_date = str(datetime.now().date())
    print(current_date, type(current_date))
    pipelines_dict = {}
    print(response.ok)
    json=response.json()
#     print(json,"\n\n")
    # print(json.get('value')[0].get('url'))
    print(json.keys())
    print(json['count'])
    # print(json.values())
    values = json['value']
    print(values[0])
    print(type(values),"\n")
    print(values[0].keys(),"\n")
    for value in values:
        today_date = datetime.strptime(current_date, "%Y-%m-%d")
        run_date = datetime.strptime(value['createdDate'][:10], "%Y-%m-%d")
        delta = today_date - run_date
        date_diff = delta.days
        if value['state'] == "completed" and date_diff <= 7:
#             print("Build Pipeline: ",value['name'])
#             print("Build Pipeline: ",value['state'])
#             print("Build Result: ",value['result'])
#             print("Build Date: ",value['createdDate'][:10])
#             print("------")
            pipelines_dict[value['id']] = [value['name'],value['createdDate'][:10],value['result']]

    return pipelines_dict

getPipelinesRuns()

Om5ic3RiZXduZnBrNW9oM2dnZXgzcXltenhoeG1qaWVraDM2Y2RnM2x4b2VmNXFjd3J5bHE=
2023-06-28 <class 'str'>
True
dict_keys(['count', 'value'])
9
{'_links': {'self': {'href': 'https://dev.azure.com/noobDevOps/26c0c59b-ecb0-4665-b746-ee087d5f30d2/_apis/pipelines/1/runs/47'}, 'web': {'href': 'https://dev.azure.com/noobDevOps/26c0c59b-ecb0-4665-b746-ee087d5f30d2/_build/results?buildId=47'}, 'pipeline.web': {'href': 'https://dev.azure.com/noobDevOps/26c0c59b-ecb0-4665-b746-ee087d5f30d2/_build/definition?definitionId=1'}, 'pipeline': {'href': 'https://dev.azure.com/noobDevOps/26c0c59b-ecb0-4665-b746-ee087d5f30d2/_apis/pipelines/1?revision=1'}}, 'templateParameters': {}, 'pipeline': {'url': 'https://dev.azure.com/noobDevOps/26c0c59b-ecb0-4665-b746-ee087d5f30d2/_apis/pipelines/1?revision=1', 'id': 1, 'revision': 1, 'name': 'First-Pipeline-yaml', 'folder': '\\'}, 'state': 'completed', 'result': 'canceled', 'createdDate': '2023-06-26T09:55:59.3967327Z', 'finishedDate': '2023-06-26T10:49:27.4947863Z', 'url

{47: ['FirstPipeline', '2023-06-26', 'canceled'],
 46: ['FirstPipeline', '2023-06-26', 'canceled'],
 45: ['FirstPipeline', '2023-06-26', 'canceled'],
 44: ['FirstPipeline', '2023-06-26', 'failed'],
 43: ['FirstPipeline', '2023-06-26', 'failed'],
 42: ['FirstPipeline', '2023-06-26', 'failed']}

# For git commits

In [38]:
import requests
import json
import base64
from bs4 import BeautifulSoup

# Azure DevOps credentials
pat = ''
organization = 'noobDevOps'
project = 'TheMcguffin'
repositoryId= 'First-Pipeline-yaml'

authorization = str(base64.b64encode(bytes(':'+pat, 'ascii')), 'ascii')

print(authorization)

url = f"https://dev.azure.com/{organization}/{project}/_apis/pipelines/1/runs?api-version=6.1-preview.1"
url = f"https://dev.azure.com/{organization}/{project}/_apis/pipelines?api-version=6.1-preview.1"
url = f"https://dev.azure.com/{organization}/{project}/_apis/git/repositories/{repositoryId}/commits?api-version=6.1"



payload = json.dumps({})
headers = {
  'Authorization': 'Basic '+authorization,
  'Content-Type': 'application/json',
  
}

response = requests.request("GET", url, headers=headers)
last7Commits = []
# print(help(response))
print(response.ok)
json=response.json()
# print(json)
# print(json.get('value')[0].get('url'))
print(json.keys())
print(json.get('count'))
commits = json['value']
for i, commit in enumerate(commits):
    print(commit)
    print(i,"--->",commit['author']['date'][:10])
# # print(json.get('state'))
# soup = BeautifulSoup(response.text, 'html.parser')
# # print(soup.prettify())

Om5ic3RiZXduZnBrNW9oM2dnZXgzcXltenhoeG1qaWVraDM2Y2RnM2x4b2VmNXFjd3J5bHE=
True
dict_keys(['count', 'value'])
24
{'commitId': '09303e9aa134e7e83f2cefe2b5ee9c4a28bf7936', 'author': {'name': 'Ashish Kandwal', 'email': 'ashish.kandwal@capgemini.com', 'date': '2022-08-02T06:57:06Z'}, 'committer': {'name': 'Ashish Kandwal', 'email': 'ashish.kandwal@capgemini.com', 'date': '2022-08-02T06:57:06Z'}, 'comment': 'Updated triggering.yaml', 'changeCounts': {'Add': 0, 'Edit': 1, 'Delete': 0}, 'url': 'https://dev.azure.com/noobDevOps/26c0c59b-ecb0-4665-b746-ee087d5f30d2/_apis/git/repositories/3d24599f-d910-4aa3-97e4-a775a127b35e/commits/09303e9aa134e7e83f2cefe2b5ee9c4a28bf7936', 'remoteUrl': 'https://dev.azure.com/noobDevOps/TheMcguffin/_git/First-Pipeline-yaml/commit/09303e9aa134e7e83f2cefe2b5ee9c4a28bf7936'}
0 ---> 2022-08-02
{'commitId': '80930bdc1c9e05f2cc26a64bbdd1f33ad95e7e9b', 'author': {'name': 'Ashish Kandwal', 'email': 'ashish.kandwal@capgemini.com', 'date': '2022-08-02T06:45:21Z'}, 'committ

# Get Pipeline Stages

In [39]:
import requests
import json
import base64
from bs4 import BeautifulSoup
from datetime import datetime


def getPipelinesRuns():
    # Azure DevOps credentials
    pat = ''
    organization = 'noobDevOps'
    project = 'TheMcguffin'

    authorization = str(base64.b64encode(bytes(':'+pat, 'ascii')), 'ascii')

    print(authorization)
    
    url = f"https://dev.azure.com/{organization}/{project}/_apis/build/builds/{47}/timeline"

#     payload = json.dumps({})
    headers = {
      'Authorization': 'Basic '+authorization,
      'Content-Type': 'application/json',
    }

    response = requests.get(url, headers=headers)
    print(response.ok)
    print(response.json()['records'][0].keys())
#     print(response.json.get('records'))
getPipelinesRuns()

Om5ic3RiZXduZnBrNW9oM2dnZXgzcXltenhoeG1qaWVraDM2Y2RnM2x4b2VmNXFjd3J5bHE=
True
dict_keys(['previousAttempts', 'id', 'parentId', 'type', 'name', 'startTime', 'finishTime', 'currentOperation', 'percentComplete', 'state', 'result', 'resultCode', 'changeId', 'lastModified', 'workerName', 'order', 'details', 'errorCount', 'warningCount', 'url', 'log', 'task', 'attempt', 'identifier'])


# Get Pull Requests

In [ ]:
import requests
import json
import base64
from bs4 import BeautifulSoup
from datetime import datetime


def getPipelinesRuns():
    # Azure DevOps credentials
    pat = ''
    organization = 'noobDevOps'
    project = 'TheMcguffin'

    authorization = str(base64.b64encode(bytes(':'+pat, 'ascii')), 'ascii')

    print(authorization)
    
    url = f"https://dev.azure.com/{organization}/{project}/_apis/build/builds/{47}/timeline"

#     payload = json.dumps({})
    headers = {
      'Authorization': 'Basic '+authorization,
      'Content-Type': 'application/json',
    }

    response = requests.get(url, headers=headers)
    print(response.ok)
    print(response.json()['records'][0].keys())
#     print(response.json.get('records'))
getPipelinesRuns()

# Using Azure Devops API

In [1]:
import datetime
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import matplotlib.pyplot as plt
 

# Azure DevOps organization URL and personal access token
organization_url = "https://dev.azure.com/noobDevOps"
personal_access_token = ""
project = 'TheMcguffin'
repositoryId= 'First-Pipeline-yaml'

# Create a connection with basic authentication using PAT
credentials = BasicAuthentication("", personal_access_token)
connection = Connection(base_url=organization_url, creds=credentials)

# Get a client for making API requests
client = connection.clients.get_build_client()

# Example: Fetching build pipeline data
builds = client.get_builds(project=project)

 

# Process the fetched data as needed
for build in builds:
    print("Build ID:", build.id)
    print("Definition ID:", build.definition.id)
    print("Status:", build.status)
    print("Result:", build.result)
    print("Start Time:", build.start_time)
    print("Finish Time:", build.finish_time)
    print("---")


# Function to fetch and process build data
def fetch_build_data():
    builds = client.get_builds(project="Pipeline", top=10, query_order="finishTimeDescending")
    
    # Initialize lists to store build IDs and completion times
    build_ids = []
    completion_times = []
    
    # Process the fetched data and extract build IDs and completion times
    for build in builds:
        build_ids.append(build.id)
        completion_times.append(build.finish_time)
    
    # Create a line chart for build completion times
    plt.figure(figsize=(8, 6))
    plt.plot(build_ids, completion_times, marker='o')
    plt.xlabel('Build ID')
    plt.ylabel('Completion Time')
    plt.title('Build Completion Times')
    
    # Add build IDs as annotations on the chart
    for i in range(len(build_ids)):
        plt.annotate(build_ids[i], xy=(build_ids[i], completion_times[i]), xytext=(5, -5),
                     textcoords='offset points', ha='left', va='bottom', fontsize=8)

    plt.show()

Build ID: 44
Definition ID: 1
Status: completed
Result: failed
Start Time: 2023-06-26 10:39:07.450218+00:00
Finish Time: 2023-06-26 11:39:13.132735+00:00
---
Build ID: 47
Definition ID: 1
Status: completed
Result: canceled
Start Time: 2023-06-26 10:49:27.494786+00:00
Finish Time: 2023-06-26 10:49:27.494786+00:00
---
Build ID: 46
Definition ID: 1
Status: completed
Result: canceled
Start Time: 2023-06-26 10:49:24.937246+00:00
Finish Time: 2023-06-26 10:49:24.937246+00:00
---
Build ID: 45
Definition ID: 1
Status: completed
Result: canceled
Start Time: 2023-06-26 10:49:22.332464+00:00
Finish Time: 2023-06-26 10:49:22.332464+00:00
---
Build ID: 43
Definition ID: 1
Status: completed
Result: failed
Start Time: 2023-06-26 09:38:47.009858+00:00
Finish Time: 2023-06-26 10:39:00.137953+00:00
---
Build ID: 42
Definition ID: 1
Status: completed
Result: failed
Start Time: 2023-06-26 08:38:31.859798+00:00
Finish Time: 2023-06-26 09:38:39.525457+00:00
---
Build ID: 41
Definition ID: 2
Status: complete

In [ ]:
import datetime

 

# Simulated data for demonstration purposes
build_pipeline_data = [
    {"timestamp": datetime.datetime(2023, 6, 13, 9, 0), "success": True, "duration": 120},
    {"timestamp": datetime.datetime(2023, 6, 13, 10, 30), "success": True, "duration": 150},
    {"timestamp": datetime.datetime(2023, 6, 13, 13, 15), "success": False, "duration": 180},
    {"timestamp": datetime.datetime(2023, 6, 13, 15, 0), "success": True, "duration": 110},
    # Add more build pipeline data here
]

def calculate_lead_time(changes):
    first_change = min(changes, key=lambda x: x['timestamp'])
    last_change = max(changes, key=lambda x: x['timestamp'])
    lead_time = last_change['timestamp'] - first_change['timestamp']
    return lead_time.total_seconds() / 60  # Convert to minutes

def calculate_deployment_frequency(changes):
    num_changes = len(changes)
    if num_changes <= 1:
        return 0
    first_change = min(changes, key=lambda x: x['timestamp'])
    last_change = max(changes, key=lambda x: x['timestamp'])
    duration = (last_change['timestamp'] - first_change['timestamp']).total_seconds()
    return num_changes / (duration / 60)  # Changes per hour

def calculate_mean_time_to_detect(pipelines):
    detected_pipelines = [pipeline for pipeline in pipelines if not pipeline['success']]
    if not detected_pipelines:
        return 0
    total_time = sum([(pipeline['timestamp'] - min(pipelines, key=lambda x: x['timestamp'])['timestamp']).total_seconds()
                      for pipeline in detected_pipelines])
    return total_time / len(detected_pipelines) / 60  # Convert to minutes

def calculate_mean_time_to_recover(pipelines):
    failed_pipelines = [pipeline for pipeline in pipelines if not pipeline['success']]
    if not failed_pipelines:
        return 0
    total_time = sum([pipeline['duration'] for pipeline in failed_pipelines])
    return total_time / len(failed_pipelines)

def calculate_build_failure_rate(pipelines):
    num_failures = len([pipeline for pipeline in pipelines if not pipeline['success']])
    return (num_failures / len(pipelines)) * 100 if pipelines else 0

def calculate_build_duration(pipelines):
    durations = [pipeline['duration'] for pipeline in pipelines]
    return sum(durations) / len(durations) if durations else 0

def calculate_first_time_build_success_rate(pipelines):
    num_first_time_successes = len([pipeline for pipeline in pipelines if pipeline['success']])
    return (num_first_time_successes / len(pipelines)) * 100 if pipelines else 0

def calculate_build_queue_length(pipelines):
    return len(pipelines)


# Calculate DORA metrics
lead_time = calculate_lead_time(build_pipeline_data)
deployment_frequency = calculate_deployment_frequency(build_pipeline_data)
mean_time_to_detect = calculate_mean_time_to_detect(build_pipeline_data)
mean_time_to_recover = calculate_mean_time_to_recover(build_pipeline_data)
build_failure_rate = calculate_build_failure_rate(build_pipeline_data)
build_duration = calculate_build_duration(build_pipeline_data)
first_time_build_success_rate = calculate_first_time_build_success_rate(build_pipeline_data)
build_queue_length = calculate_build_queue_length(build_pipeline_data)


# Print DORA metrics
print("Lead Time for Changes:", lead_time, "minutes")
print("Deployment Frequency:", deployment_frequency, "changes per hour")
print("Mean Time to Detect:", mean_time_to_detect, "minutes")
print("Mean Time to Recover:", mean_time_to_recover, "minutes")
print("Build Failure Rate:", build_failure_rate, "%")
print("Build Duration:", build_duration, "minutes")
print("First-Time Build Success Rate:", first_time_build_success_rate, "%")
print("Build Queue Length:", build_queue_length)